In [ ]:
# CELL 0 – Imports and physical constants

import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u   # ← THIS is what was missing
from scipy.interpolate import griddata

# Physical constants
G   = 6.67430e-11         # m^3 kg^-1 s^-2
c   = 2.99792458e8        # m/s
M_sun = 1.98847e30        # kg
kpc = 3.085677581e19      # m

# (optional) cosmology object if you need it later
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [ ]:
# ================================================================
# CELL 1 — Lens configuration input block for HE 0435−1223
# ================================================================
# Change ONLY this cell for each lens.

# 1. Redshifts (lens & source)
# Lens redshift and source redshift from H0LiCOW / COSMOGRAIL
z_d = 0.4546   # lens galaxy
z_s = 1.693    # quasar source

# 2. Angular-diameter distances using CELL 0 cosmology
D_d  = cosmo.angular_diameter_distance(z_d)            # [Mpc]
D_s  = cosmo.angular_diameter_distance(z_s)            # [Mpc]
D_ds = cosmo.angular_diameter_distance_z1z2(z_d, z_s)  # [Mpc]

# Convert to metres and build time-delay distance factor
D_d_m  = D_d.to(u.m).value
D_s_m  = D_s.to(u.m).value
D_ds_m = D_ds.to(u.m).value

# Time-delay distance factor: (1+z_d) D_d D_s / D_ds   [metres]
time_delay_distance_factor = (1.0 + z_d) * (D_d_m * D_s_m / D_ds_m)
print(f"(1+z_d) D_d D_s / D_ds = {time_delay_distance_factor:.3e} m")

# 3. Einstein radius (arcsec)
# Effective Einstein radius from HST lens modelling (H0LiCOW IV, approx.)
theta_E_arcsec = 1.18

# 4. Lens mass ellipticity and orientation
# HE 0435 lens is an almost round early-type; q ~ 0.8 is a good approximation.
axis_ratio_q = 0.80          # b/a
position_angle_phi = -15.0   # degrees East of North (approximate major-axis PA)

# 5. External shear (environment)
# HE 0435 has a modest group environment; gamma_ext ~ 0.05 is typical of SIE+γ fits.
gamma_ext     = 0.05         # shear amplitude (dimensionless, approximate)
phi_gamma_deg = 45.0         # shear angle in degrees (approximate)

# 6. External convergence (LOS mass-sheet)
# Line-of-sight environment is close to cosmic mean; κ_env is small but non-zero.
# Use κ_env ≈ 0.04–0.08 in detailed work; we start with 0.05.
kappa_env = 0.05

# 7. Image positions (arcsec, lens-centered)
# These are approximate HST-like positions in lens-centred coordinates.
# For a fully precise run, replace with the exact astrometry table,
# but these are good enough for the SFH 10-lens survey.
image_positions = {
    "A": ( +1.18,  0.00 ),   # image A near +θ_E on x-axis
    "B": ( -0.90, +0.95 ),   # image B
    "C": ( -1.15, -0.20 ),   # image C
    "D": ( +0.90, -0.90 )    # image D
}

# 8. MGE parameters (shape only — arbitrary normalization fixed by θ_E)
# This is a “generic” 10-Gaussian MGE for an L* elliptical at this redshift.
# For full realism, these can be replaced by the exact H0LiCOW MGE coefficients,
# but this form is consistent with the J1206 / PG1115 template.
MGE_sigmas_arcsec = [
    0.06, 0.12, 0.20, 0.35, 0.55,
    0.80, 1.15, 1.70, 2.40, 3.80
]

MGE_amps = [
    7.0, 6.0, 4.8, 3.5, 2.4,
    1.6, 1.0, 0.6, 0.3, 0.12
]

(1+z_d) D_d D_s / D_ds = 8.507e+25 m


In [ ]:
# CELL 2 — Build grid & MGE model (does not change)

arcsec_to_rad = (np.pi/180)/3600

x_vals_arcsec = np.linspace(-5, 5, 500)
y_vals_arcsec = np.linspace(-5, 5, 500)
x_grid_arcsec, y_grid_arcsec = np.meshgrid(x_vals_arcsec, y_vals_arcsec)

def elliptical_gaussian(x, y, sigma, amp, q, phi_deg):
    phi = np.deg2rad(phi_deg)
    cosp, sinp = np.cos(phi), np.sin(phi)
    x_rot =  cosp*x + sinp*y
    y_rot = -sinp*x + cosp*y
    r2 = x_rot**2 + (y_rot/q)**2
    return amp*np.exp(-0.5*r2/sigma**2)

def psi_norm(x, y):
    psi = np.zeros_like(x)
    for sigma, amp in zip(MGE_sigmas_arcsec, MGE_amps):
        psi += elliptical_gaussian(
            x, y, sigma, amp,
            axis_ratio_q, position_angle_phi
        )
    return psi

psi_mass_norm = psi_norm(x_grid_arcsec, y_grid_arcsec)

In [ ]:
# CELL 3 — Normalize ψ via |∇ψ| = θ_E (does not change)

# Gradients
dpsi_dy_arcsec, dpsi_dx_arcsec = np.gradient(
    psi_mass_norm,
    y_vals_arcsec,
    x_vals_arcsec
)

# convert to radian deflection
dpsi_dx_rad = dpsi_dx_arcsec / arcsec_to_rad
dpsi_dy_rad = dpsi_dy_arcsec / arcsec_to_rad
alpha_mag = np.sqrt(dpsi_dx_rad**2 + dpsi_dy_rad**2)

# Einstein ring mask
theta_E_rad = theta_E_arcsec * arcsec_to_rad
r_grid = np.sqrt(x_grid_arcsec**2 + y_grid_arcsec**2)
ring_mask = np.abs(r_grid - theta_E_arcsec) <= 0.1

mean_defl_norm = alpha_mag[ring_mask].mean()
A_scale = theta_E_rad / mean_defl_norm

psi_mass_phys = psi_mass_norm * A_scale

In [ ]:
# CELL 4 — Add external shear (does not change)

phi_g = np.deg2rad(phi_gamma_deg)
cos2phi = np.cos(2*phi_g)
sin2phi = np.sin(2*phi_g)

x_rad = x_grid_arcsec * arcsec_to_rad
y_rad = y_grid_arcsec * arcsec_to_rad

psi_shear = 0.5*gamma_ext * (
    (x_rad**2 - y_rad**2)*cos2phi +
    2*x_rad*y_rad*sin2phi
)

psi_total = psi_mass_phys + psi_shear

In [ ]:
# ================================================================
# CELL 5 — Evaluate ψ_SFH at ALL image positions & compute all Δt
# ================================================================

def psi_at(theta_x, theta_y):
    """Interpolate ψ_SFH at an image position."""
    return griddata(
        (x_grid_arcsec.ravel(), y_grid_arcsec.ravel()),
        psi_total.ravel(),
        (theta_x, theta_y),
        method='cubic'
    )

# --- 1. Compute ψ for every image ---
psi_values = {}
for label, (x, y) in image_positions.items():
    psi_values[label] = psi_at(x, y)

print("ψ values at image positions:")
for k, v in psi_values.items():
    print(f"  ψ_{k} = {v:.6e}")
print()

# --- 2. Compute all pairwise delays ---
labels = list(image_positions.keys())
print("Pairwise SFH Time Delays:\n")

for i in range(len(labels)):
    for j in range(i+1, len(labels)):
        Li, Lj = labels[i], labels[j]
        psi_i = psi_values[Li]
        psi_j = psi_values[Lj]

        dpsi = psi_j - psi_i
        dt_raw = (dpsi * time_delay_distance_factor / c) / (24*3600)
        dt_corr = dt_raw / (1 - kappa_env)

        print(f"{Lj} – {Li}:")
        print(f"   Δψ = {dpsi:.6e}")
        print(f"   Raw SFH Δt = {dt_raw:.2f} days")
        print(f"   κ_env-corrected = {dt_corr:.2f} days\n")

ψ values at image positions:
  ψ_A = 2.372542e-11
  ψ_B = 1.754227e-11
  ψ_C = 2.334582e-11
  ψ_D = 1.862947e-11

Pairwise SFH Time Delays:

B – A:
   Δψ = -6.183146e-12
   Raw SFH Δt = -20.31 days
   κ_env-corrected = -21.38 days

C – A:
   Δψ = -3.796036e-13
   Raw SFH Δt = -1.25 days
   κ_env-corrected = -1.31 days

D – A:
   Δψ = -5.095951e-12
   Raw SFH Δt = -16.74 days
   κ_env-corrected = -17.62 days

C – B:
   Δψ = 5.803543e-12
   Raw SFH Δt = 19.06 days
   κ_env-corrected = 20.06 days

D – B:
   Δψ = 1.087196e-12
   Raw SFH Δt = 3.57 days
   κ_env-corrected = 3.76 days

D – C:
   Δψ = -4.716347e-12
   Raw SFH Δt = -15.49 days
   κ_env-corrected = -16.30 days

